In [1]:
import importlib
import dill as pickle
from tqdm import tqdm
from tqdm import trange

import pentago
from pentago import agent, memory, minimax, learning_agent, neural_model, monte_carlo, evaluate, game, multi

import ray
ray.init()

2019-04-16 13:33:34,345	WARNING worker.py:1406 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-04-16 13:33:34,348	INFO node.py:423 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-16_13-33-34_20162/logs.
2019-04-16 13:33:34,457	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:50590 to respond...


2019-04-16 13:33:34,573	INFO services.py:363 -- Waiting for redis server at 127.0.0.1:20172 to respond...
2019-04-16 13:33:34,576	INFO services.py:760 -- Starting Redis shard with 6.87 GB max memory.
2019-04-16 13:33:34,591	INFO services.py:1384 -- Starting the Plasma object store with 10.31 GB memory using /tmp.
2019-04-16 13:33:34,604	WARNING services.py:863 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': None,
 'redis_address': '10.110.8.99:50590',
 'object_store_address': '/tmp/ray/session_2019-04-16_13-33-34_20162/sockets/plasma_store',
 'webui_url': None,
 'raylet_socket_name': '/tmp/ray/session_2019-04-16_13-33-34_20162/sockets/raylet'}

In [2]:
view = memory.MemoryView()
round = 0

In [3]:
multi.self_play_games(minimax.MinimaxSearchAgent(depth=2),
                      view,
                      1000,
                      verbose=True,
                      num_workers=20)

646.0712208747864 seconds taken, 0.6460712208747864 per game


In [ ]:
pickle.dump(view, open('initial_view.pkl', mode='wb'))

In [ ]:
view = pickle.load(open('initial_view.pkl', mode='rb'))

In [ ]:
model_params = {'dense_regularization_const': 1e-2,
                'regularization_const': 1e-4,
                'num_layers': 5,
                'kernel_size': 5,
                'num_filters': 64}
current_agent = learning_agent.NeuralAgent(model_params=model_params)
best_agent = learning_agent.NeuralAgent(model_params=model_params)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
while True:
    round += 1
    multi.self_play_games(current_agent,
                          view,
                          100,
                          verbose=True,
                          num_workers=20)
    for e in range(4):
        current_agent.fit(view.get_experiences(sample_k=1000), epochs=1)
    if round % 5 == 0:
        wins = multi.tally_wins([minimax.MinimaxSearchAgent(), current_agent], 10, num_workers=3)
        print('Round {}, against minimax: {}'.format(round, wins))
        wins = multi.tally_wins([best_agent, current_agent], 10, num_workers=3)
        print('Round {}, against current best neural: {}'.format(round, wins))
        if wins[-1] > 7:
            print('Ratcheting best agent')
            best_agent.model.set_weights(current_agent.model.get_weights())

(pid=20189) WARNING:tensorflow:From /anaconda3/envs/gai-final-project/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=20189) Instructions for updating:
(pid=20189) Colocations handled automatically by placer.
(pid=20190) WARNING:tensorflow:From /anaconda3/envs/gai-final-project/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=20190) Instructions for updating:
(pid=20190) Colocations handled automatically by placer.
(pid=20185) WARNING:tensorflow:From /anaconda3/envs/gai-final-project/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=20185) Instructions for updat

(pid=20193) WARNING:tensorflow:From /anaconda3/envs/gai-final-project/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=20193) Instructions for updating:
(pid=20193) Colocations handled automatically by placer.
(pid=20194) WARNING:tensorflow:From /anaconda3/envs/gai-final-project/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=20194) Instructions for updating:
(pid=20194) Colocations handled automatically by placer.
(pid=20191) WARNING:tensorflow:From /anaconda3/envs/gai-final-project/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=20191) Instructions for updat

In [ ]:
# some kind of epic memory leak going on...
view.temp_memory.clear()
ray.shutdown()
importlib.reload(multi)
ray.init()

In [ ]:
while True:
    round += 1
    multi.self_play_games(current_agent,
                          view,
                          100,
                          verbose=True,
                          num_workers=20)
    for e in range(4):
        current_agent.fit(view.get_experiences(sample_k=1000), epochs=1)
    if round % 5 == 0:
        wins = multi.tally_wins([minimax.MinimaxSearchAgent(), current_agent], 10, num_workers=3)
        print('Round {}, against minimax: {}'.format(round, wins))
        wins = multi.tally_wins([best_agent, current_agent], 10, num_workers=3)
        print('Round {}, against current best neural: {}'.format(round, wins))
        if wins[-1] > 7:
            print('Ratcheting best agent')
            best_agent.model.set_weights(current_agent.model.get_weights())